# AIML339 Assignment 3
## Matrix Profiles for Anomaly Detection and Time Series Dynamics

The matrix profile is primarily used to find repeating patterns and outliers in a single, univariate time series.. This is based on the principle of similarity joins - comparing a sub-sequence of the time series against itself, by computing the distance between each pair of subsequences. Matrix profile algorithms have the following properties:
- Domain agnostic
- Can be exact or approximate
- Only one hyperparameter (window length)
- Parallelizable
- Stream compatible
- Handles missing data

## Terms:
- **Motif**: A repeated pattern in the time series.
- **Discord**: An anomaly in the time series.


